In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 5.8 MB 64.4 MB/s 
     |████████████████████████████████| 1.3 MB 58.1 MB/s 
     |████████████████████████████████| 182 kB 77.1 MB/s 
     |████████████████████████████████| 7.6 MB 67.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8d8e32f7d22e49d7e5de4e24c6082eff5a6214ab47ed21004da9bb1e32b16c86
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
#its better to connect without gpu

In [ ]:
major = []
d = {}
with open("/content/drive/MyDrive/Prof. Asim Tewari/test (1).txt") as f:
  temp = ""
  i = 0
  for line in f:
    if line.startswith(" ")  or line.startswith("<end>"):
        
        d["chapter_no"] = 2
        d["para_id"] = i
        d["content"] = temp
        
        major.append(d)
        temp = line
        i = i + 1
        d = {}
        continue

    temp = temp + line


def func(value):         #to remove \n formatting character from the text
    return ' '.join(value.splitlines())

for dic in major:
   dic["content"] = func(dic["content"])

In [ ]:
qa = major.copy()

In [ ]:
major

[{'chapter_no': 2,
  'para_id': 0,
  'content': 'We know about people who lived in the subcontinent as early as two million years ago. Today, we describe them as hunter-gatherers. The name comes from the way in which they got their food. Generally, they hunted wild animals, caught fish and birds, gathered fruits, roots, nuts, seeds, leaves, stalks and eggs.'},
 {'chapter_no': 2,
  'para_id': 1,
  'content': '  Hunter-gatherers moved from place to place. There are many reasons for this.'},
 {'chapter_no': 2,
  'para_id': 2,
  'content': '  First, if they had stayed at one place for a long time, they would have eaten up all the available plant and animal resources. Therefore, they would have had to go elsewhere in search of food.'},
 {'chapter_no': 2,
  'para_id': 3,
  'content': '  Second, animals move from place to place — either in search of smaller prey, or, in the case of deer and wild cattle, in search of grass and leaves. That is why those who hunted them had to follow their movem

In [ ]:
for i in qa:
   i["encoding"] = model.encode(i['content']).tolist()

In [ ]:
!pip install pinecone-client
import pinecone

pinecone.init(
    api_key= "c77cdf54-33f7-4ddb-b8cd-284af704b5b2" ,
    environment='us-west1-gcp'
)

# check if index already exists, if not we create it
if 'qa-index' not in pinecone.list_indexes():
    pinecone.create_index(
        name='qa-index',
        dimension=len(qa[0]['encoding'])
    )

# connect to index
index = pinecone.Index('qa-index')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 29.9 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 


In [ ]:
from tqdm.auto import tqdm  # progress bar

upserts = [(str(v['para_id']), v['encoding'],{"type": "para"}) for v in qa]
# now upsert in chunks
for i in tqdm(range(0, len(upserts), 1)):
    i_end = i + 1
    if i_end > len(upserts): 
      i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

  0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:

query = "Explain the beginning of farming and herding"
xq = model.encode([query]).tolist()

In [ ]:
xc = index.query(xq,
                 filter = { 
                     "type" : {"$eq": "para"}
                 },
                 top_k=3)
xc


{'matches': [{'id': '19',
              'score': 0.569713295,
              'sparseValues': {},
              'values': []},
             {'id': '21',
              'score': 0.535790503,
              'sparseValues': {},
              'values': []},
             {'id': '20',
              'score': 0.51480335,
              'sparseValues': {},
              'values': []}],
 'namespace': ''}

In [ ]:
ids = [x['id'] for x in xc['matches']]
contexts = [qa[i]["content"] for i in range(len(qa)) if str(qa[i]["para_id"]) in ids]
# contexts = {"context":[]}
# for i in range(len(qa)):
#   if qa[i]["para_id"] in ids:
#     contexts["content"].append(qa[i]["content"])


In [ ]:
contexts

['  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons. It is likely that this helped people to start thinking about herding and rearing these animals themselves. Fishing also became important.',
 '  This was also a time when several grain bearing grasses, including wheat, barley and rice grew naturally in different parts of the subcontinent. Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened. This may have led them to think about growing plants on their own. In this way people became farmers.',
 '  People could also attract and then tame animals by leaving food for them near their shelters. The first animal to be tamed was the wild ancestor of the dog. Later, people encouraged animals that were relatively gentle to come near the camps where they lived. These animals such as sheep, goat, cattle and also the pig lived in herds, and most of them ate grass. Often, peo

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Story of Sentences begun

In [ ]:
sen = []

sentdata = []
for sent in contexts:
  sen = sen + sent_tokenize(sent)
for i in range(len(sen)):
  temp = {}
  temp["chapter_no"] = 2
  temp["sen_id"] = i
  temp["sen_con"] = sen[i] 
  sentdata.append(temp)
 
qa1 = sentdata.copy()

In [ ]:
sentdata

[{'chapter_no': 2,
  'sen_id': 0,
  'sen_con': '  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons.'},
 {'chapter_no': 2,
  'sen_id': 1,
  'sen_con': 'It is likely that this helped people to start thinking about herding and rearing these animals themselves.'},
 {'chapter_no': 2, 'sen_id': 2, 'sen_con': 'Fishing also became important.'},
 {'chapter_no': 2,
  'sen_id': 3,
  'sen_con': '  This was also a time when several grain bearing grasses, including wheat, barley and rice grew naturally in different parts of the subcontinent.'},
 {'chapter_no': 2,
  'sen_id': 4,
  'sen_con': 'Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened.'},
 {'chapter_no': 2,
  'sen_id': 5,
  'sen_con': 'This may have led them to think about growing plants on their own.'},
 {'chapter_no': 2,
  'sen_id': 6,
  'sen_con': 'In this way people became farmers.'},
 {'chapter_no': 2,
  'sen_id': 

In [ ]:
for i in qa1:
   i["encoding"] = model.encode(i['sen_con']).tolist()  

In [ ]:
pinecone.init(
    api_key= "c77cdf54-33f7-4ddb-b8cd-284af704b5b2" ,
    environment='us-west1-gcp'
)

# check if index already exists, if not we create it
if 'qa-index' not in pinecone.list_indexes():
    pinecone.create_index(
        name='qa-index',
        dimension=len(qa1[0]['encoding'])
    ) 

# connect to index
index = pinecone.Index('qa-index')

In [ ]:
from tqdm.auto import tqdm  # progress bar

upserts = [(str(v['sen_id']), v['encoding'],{"type":"sentence"}) for v in qa1]
# now upsert in chunks
for i in tqdm(range(0, len(upserts), 1)):
    i_end = i + 1
    if i_end > len(upserts): 
      i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:

query = "Explain the beginning of farming and herding"
xq1 = model.encode([query]).tolist()

In [ ]:
xc1 = index.query(xq1,
                  filter = {
                          "type":{"$eq":"sentence"}
                  },
                  top_k=6)
xc1


{'matches': [{'id': '6',
              'score': 0.633554935,
              'sparseValues': {},
              'values': []},
             {'id': '12',
              'score': 0.60747093,
              'sparseValues': {},
              'values': []},
             {'id': '1',
              'score': 0.537129402,
              'sparseValues': {},
              'values': []},
             {'id': '0',
              'score': 0.460388422,
              'sparseValues': {},
              'values': []},
             {'id': '4',
              'score': 0.433204025,
              'sparseValues': {},
              'values': []},
             {'id': '10',
              'score': 0.405087352,
              'sparseValues': {},
              'values': []}],
 'namespace': ''}

In [ ]:
ids1 = [x['id'] for x in xc1['matches']]
contexts1 = [qa1[i]["sen_con"] for i in range(len(qa1)) if str(qa1[i]["sen_id"]) in ids1]  
# contexts = {"context":[]}
# for i in range(len(qa)):
#   if qa[i]["para_id"] in ids:
#     contexts["content"].append(qa[i]["content"])


In [ ]:
contexts1

['  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons.',
 'It is likely that this helped people to start thinking about herding and rearing these animals themselves.',
 'Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened.',
 'In this way people became farmers.',
 'These animals such as sheep, goat, cattle and also the pig lived in herds, and most of them ate grass.',
 'This is how they became herders.']

In [ ]:
contexts   #Explain the beginning of farming and herding

['  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons. It is likely that this helped people to start thinking about herding and rearing these animals themselves. Fishing also became important.',
 '  This was also a time when several grain bearing grasses, including wheat, barley and rice grew naturally in different parts of the subcontinent. Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened. This may have led them to think about growing plants on their own. In this way people became farmers.',
 '  People could also attract and then tame animals by leaving food for them near their shelters. The first animal to be tamed was the wild ancestor of the dog. Later, people encouraged animals that were relatively gentle to come near the camps where they lived. These animals such as sheep, goat, cattle and also the pig lived in herds, and most of them ate grass. Often, peo

In [ ]:
! pip install sentence-splitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 3.7 MB/s 


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

! pip install SentencePiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model1 = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model1.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [ ]:
text = "Those who hunted these animals now followed them, learning about their food habits and their breeding seasons."

In [ ]:
get_response(text, 5)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['Those who hunted these animals learned about their food habits and breeding seasons.',
 'People who hunted these animals learned about their food habits and breeding seasons.',
 'The people who hunted these animals learned about their food habits and breeding seasons.',
 'Those who hunted these animals learned a lot about their food and breeding seasons.',
 'Those who hunted these animals learned a lot about their food habits and breeding seasons.']

In [ ]:
get_response(text, 1)

['Those who hunted these animals learned about their food habits and breeding seasons.']

In [ ]:
# Paragraph of text

context = " ".join(contexts1)


In [ ]:
# # Takes the input paragraph and splits it into a list of sentences
# from sentence_splitter import SentenceSplitter, split_text_into_sentences

# splitter = SentenceSplitter(language='en')

# sentence_list = splitter.split(context)
sentence_list = contexts1.copy()

In [ ]:
sentence_list

['  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons.',
 'It is likely that this helped people to start thinking about herding and rearing these animals themselves.',
 'Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened.',
 'In this way people became farmers.',
 'These animals such as sheep, goat, cattle and also the pig lived in herds, and most of them ate grass.',
 'This is how they became herders.']

In [ ]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

In [ ]:
# This is the paraphrased text
paraphrase

[['Those who hunted these animals learned about their food habits and breeding seasons.'],
 ['It is possible that this helped people to think about rearing their own animals.'],
 ['Men, women and children probably collected these grains as food and learned where they came from.'],
 ['People became farmers.'],
 ['Most of the animals that lived in herds ate grass.'],
 ['They became herders here.']]

In [ ]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['Those who hunted these animals learned about their food habits and breeding seasons.',
 'It is possible that this helped people to think about rearing their own animals.',
 'Men, women and children probably collected these grains as food and learned where they came from.',
 'People became farmers.',
 'Most of the animals that lived in herds ate grass.',
 'They became herders here.']

In [ ]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'Those who hunted these animals learned about their food habits and breeding seasons. It is possible that this helped people to think about rearing their own animals. Men, women and children probably collected these grains as food and learned where they came from. People became farmers. Most of the animals that lived in herds ate grass. They became herders here.'

In [ ]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(context)
print(paraphrased_text)

  Those who hunted these animals now followed them, learning about their food habits and their breeding seasons. It is likely that this helped people to start thinking about herding and rearing these animals themselves. Men, women and children probably collected these grains as food, and learnt where they grew, and when they ripened. In this way people became farmers. These animals such as sheep, goat, cattle and also the pig lived in herds, and most of them ate grass. This is how they became herders.
Those who hunted these animals learned about their food habits and breeding seasons. It is possible that this helped people to think about rearing their own animals. Men, women and children probably collected these grains as food and learned where they came from. People became farmers. Most of the animals that lived in herds ate grass. They became herders here.


In [ ]:
def lfqa(text,query,k):

    major = []
    d = {}
    with open(text) as f:
        temp = ""
        i = 0
        for line in f:
              if line.startswith(" ")  or line.startswith("<end>"):

                d["chapter_no"] = 2
                d["para_id"] = i
                d["content"] = temp

                major.append(d)
                temp = line
                i = i + 1
                d = {}
                continue

        temp = temp + line


    def func(value):         #to remove \n formatting character from the text
        return ' '.join(value.splitlines())

    for dic in major:
        dic["content"] = func(dic["content"])
    
    #encoding___________________________________________________________________________________________________
    
    for i in qa:
        i["encoding"] = model.encode(i['content']).tolist()
    
    #Upserting to index_________________________________________________________________________________________
    
    upserts = [(str(v['para_id']), v['encoding'],{"type": "para"}) for v in qa]
    # now upsert in chunks
    for i in tqdm(range(0, len(upserts), 1)):
        i_end = i + 1
        if i_end > len(upserts): 
            i_end = len(upserts)
        index.upsert(vectors=upserts[i:i_end])
    

    #Encoding the query__________________________________________________________________________________________
    xq = model.encode([query]).tolist()
    
    #Fetching Data from index in para-form
    xc = index.query(xq,
                 filter = { 
                     "type" : {"$eq": "para"}
                 },
                 top_k=3)
    
    ids = [x['id'] for x in xc['matches']]
    contexts = [qa[i]["content"] for i in range(len(qa)) if str(qa[i]["para_id"]) in ids]
    
    #Converting the Para data in sentence form___________________________________________________________________
    sen = []

    sentdata = []
    for sent in contexts:
        sen = sen + sent_tokenize(sent)
    for i in range(len(sen)):
        temp = {}
        temp["chapter_no"] = 2
        temp["sen_id"] = i
        temp["sen_con"] = sen[i] 
        sentdata.append(temp)
            
    qa1 = sentdata.copy()
    
    #Encoding of sentences________________________________________________________________________________________
    for i in qa1:
        i["encoding"] = model.encode(i['sen_con']).tolist()  
    
    upserts = [(str(v['sen_id']), v['encoding'],{"type":"sentence","asked_for":query}) for v in qa1]
    # now upsert in chunks
    for i in tqdm(range(0, len(upserts), 1)):
        i_end = i + 1
        if i_end > len(upserts): 
            i_end = len(upserts)
        index.upsert(vectors=upserts[i:i_end])
        
    #Fetching Data from Index_____________________________________________________________________________________
    xc1 = index.query(xq,
                  filter = {
                        "$and": [
                                {"type": {"$eq": "sentence"}},
                                {"asked_for": {"$eq": query}}
                ]
                               },
                  top_k=k)
    
    #Fetching and storing relevant contexts in para-form__________________________________________________________
    ids1 = [x['id'] for x in xc1['matches']]
    contexts1 = [qa1[i]["sen_con"] for i in range(len(qa1)) if str(qa1[i]["sen_id"]) in ids1]  
    
    
    #Answer Rewriting_____________________________________________________________________________________________
    sentence_list = contexts1.copy()
     
    paraphrase = []

    for i in sentence_list:
        a = get_response(i,1)
        paraphrase.append(a)

    paraphrase2 = [' '.join(x) for x in paraphrase]
    
    paraphrase3 = [' '.join(x for x in paraphrase2) ]
    paraphrased_text = str(paraphrase3).strip('[]').strip("'")

    return str(paraphrased_text)


In [ ]:
def original(text_path,query,no_of_sentences):
    temp = lfqa(text_path,query,no_of_sentences)
    return temp

In [ ]:
original("/content/drive/MyDrive/Prof. Asim Tewari/test (1).txt","Explain the beginning of farming and herding",6)

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

'Those who hunted these animals learned about their food habits and breeding seasons. It is possible that this helped people to think about rearing their own animals. Men, women and children probably collected these grains as food and learned where they came from. People became farmers. Most of the animals that lived in herds ate grass. They became herders here.'

In [2]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 K

In [3]:
import gradio as gr

In [ ]:
iface = gr.Interface(
    original,
    ["file",
     "text",
     gr.Slider(minimum = 3,maximum = 6,step = 1)] ,
    outputs = 'text'    
)

In [ ]:
iface.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://ecf615e9678dbb03.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
